In [25]:
import ifcopenshell
import time

#INPUT:
in_path = r"C:\Users\eibre\OneDrive - Norconsult Group\Documents\NB3_RIBarmering.ifc"
out_path = in_path
pset_name = 'IFC Armering'
property_name = 'Rebar Number'
colours = [
    [255, 255, 0], [0, 255, 255], [255, 0, 255], [0, 0, 255], [255, 153, 0], [255, 0, 0], [0, 204, 255], [153, 204, 0], [255, 102, 0], [255, 0, 255], [128, 0, 0], [128, 128, 0], [0, 128, 0], [0, 128, 128], [0, 0, 128], [128, 0, 128], [255, 128, 128], [255, 255, 153], [51, 153, 102], [153, 204, 255], [255, 153, 0], [51, 102, 255], [51, 153, 102], [255, 204, 153], [255, 255, 0], [0, 0, 255], [153, 153, 255], [51, 204, 204], [128, 0, 128], [0, 128, 128], [128, 128, 0], [255, 128, 128], [51, 102, 255], [255, 153, 204], [255, 204, 153], [128, 0, 128], [0, 128, 0], [255, 204, 0], [51, 102, 255], [51, 102, 255], [0, 255, 0], [153, 153, 255], [255, 255, 0], [255, 0, 255], [51, 204, 204], [51, 102, 255], [204, 204, 255], [51, 102, 255], [204, 255, 204], [255, 0, 0], [0, 0, 255], [51, 153, 102], [255, 255, 0], [255, 153, 0], [153, 204, 255], [255, 102, 0], [204, 153, 255], [204, 204, 255], [255, 153, 204], [51, 51, 153], [255, 255, 0], [0, 0, 255], [153, 153, 255], [51, 204, 204], [128, 0, 128], [0, 128, 128], [128, 128, 0], [255, 128, 128], [51, 102, 255], [255, 153, 204], [255, 0, 0], [0, 0, 255], [51, 153, 102], [255, 255, 0], [255, 153, 0], [153, 204, 255], [255, 102, 0], [204, 153, 255], [204, 204, 255], [255, 153, 204]
]

# Script starts:
t_start = time.perf_counter()

print(f"Opening file at {in_path}")

f = ifcopenshell.open(in_path)
rebars = f.by_type('IfcReinforcingBar')

def get_propery_value(rebar, pset_name, p_name):
    for pset in rebar.IsDefinedBy:
        if pset.RelatingPropertyDefinition.Name == pset_name:
            for property in pset.RelatingPropertyDefinition.HasProperties:
                if property.Name == p_name:
                    return property.NominalValue[0]

import collections
from collections import defaultdict
rebar_dict = defaultdict(list)
print(f"Grouping rebar by values of property {property_name} in pset {pset_name}")
for k,v in zip([int(get_propery_value(rebar, pset_name, property_name)) for rebar in rebars], rebars):
    rebar_dict[k].append(v)
    
ordered_rebars = collections.OrderedDict(sorted(rebar_dict.items()))
num_groups = len(ordered_rebars)
print(f"Found {num_groups} different groups")

import math
colours = colours*math.ceil(num_groups/len(colours))
colours = colours[:num_groups]
rgb_colours = list()

for rgb in colours:
    rgb_colours.append(f.createIfcColourRgb(None, rgb[0]/255, rgb[1]/255, rgb[2]/255))

i=0
print("Creating styled items for rebar color..")
for k, group in ordered_rebars.items():
    colour = rgb_colours[i]
    print(f"Group where {property_name} = {k} contains {len(group)} elements")
    surfaceStyleShading = f.createIfcSurfaceStyleShading()
    surfaceStyleShading.SurfaceColour = colour
    surfaceStyle = f.createIfcSurfaceStyle(colour.Name, "BOTH",(surfaceStyleShading,))
    presStyleAssign = f.createIfcPresentationStyleAssignment((surfaceStyle,))
    i=i+1
    for rebar in group:
        item = rebar.Representation.Representations[0].Items[0]
        f.createIfcStyledItem(item, (presStyleAssign,), None)
        
print(f"Saving file to {out_path}..")
f.write(out_path)
t_end = time.perf_counter()
print(f"File saved, script completed in in {t_end-t_start:0.4f} seconds")

print("Running Solibri Optimizer..")
import subprocess
dir_path = r"X:\nor\oppdrag\Trondheim\520\31\5203127\BIM\Konstruksjon\IFC"
subprocess.check_call([r'C:\Program Files\Solibri\IFCOptimizer\Solibri IFC Optimizer.exe', '-in=' + in_path, '-out=' + dir_path, '-ifc', '-force'])

import os
file_clean_name = dir_path+"\\"+os.path.basename(in_path)
file_optimized = file_clean_name[:-4]+"_optimized.ifc"
print(f"Renaming {file_optimized} to {file_clean_name}")
os.replace(file_optimized, file_clean_name)
print("Done!")

Opening file at C:\Users\eibre\OneDrive - Norconsult Group\Documents\NB3_RIBarmering.ifc
Grouping rebar by values of property Rebar Number in pset IFC Armering
Found 83 different groups
Creating styled items for rebar color..
Group where Rebar Number = 1 contains 9211 elements
Group where Rebar Number = 2 contains 5229 elements
Group where Rebar Number = 3 contains 3025 elements
Group where Rebar Number = 4 contains 1947 elements
Group where Rebar Number = 5 contains 2491 elements
Group where Rebar Number = 6 contains 1939 elements
Group where Rebar Number = 7 contains 1311 elements
Group where Rebar Number = 8 contains 1975 elements
Group where Rebar Number = 9 contains 2132 elements
Group where Rebar Number = 10 contains 2399 elements
Group where Rebar Number = 11 contains 2001 elements
Group where Rebar Number = 12 contains 1405 elements
Group where Rebar Number = 13 contains 1471 elements
Group where Rebar Number = 14 contains 623 elements
Group where Rebar Number = 15 contains 464

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\Users\\eibre\\OneDrive - Norconsult Group\\Documents\\NB3_RIBarmering_optimized.ifc' -> 'X:\\nor\\oppdrag\\Trondheim\\520\\31\\5203127\\BIM\\Konstruksjon\\IFC\\NB3_RIBarmering.ifc'